In [ ]:
from typing import Callable, List
from math import sqrt
import math


# Funções básicas

## Calculo do erro

In [ ]:
def erro_absoluto(referencia: float, estimativa: float) -> float:
    return abs(referencia - estimativa)


def erro_relativo(referencia: float, estimativa: float) -> float:
    return abs((referencia, estimativa) / estimativa)


# Derivação numerica

In [ ]:
DELTA_DERIVACAO_PADRAO = 10**-4


def derivada_direita(f: Callable[[float], float], delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x)) / delta


def derivada_esquerda(f: Callable[[float], float], delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x - delta) - f(x)) / delta


def derivada_central(f: Callable[[float], float], delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x - delta)) / (2 * delta)


# Métodos iterativos

## Condições de parada

In [ ]:
MAX_ITER_PADRAO = 100


def limite_iteracao(i: int):
    return lambda est: est.i >= i


def criterio_erro_absoluto(referencia: float, erro_max: float):
    return lambda est: erro_absoluto(referencia, est.x) < erro_max


def criterio_ab(erro_max: float):
    return lambda est: est.b - est.a < 2 * erro_max


def criterio_x(erro_max: float):
    return lambda est: est.f(est.x - erro_max) * est.f(est.x + erro_max) < 0


def criterio_delta_x(delta: float):
    return lambda est: est.prior_x is not None and abs(est.x - est.prior_x) < delta


def criterio_y(dist_zero: float):
    return lambda est: abs(est.y) < dist_zero


## Método da bisseçao

In [ ]:
class EstatisticaAB:
    def __init__(
        self, f: Callable[[float], float], a: float, b: float, x: float, y: float
    ) -> None:
        self.f = f
        self.i: int = 1
        self.a = a
        self.b = b
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, a: float, b: float, x: float, y: float):
        self.i += 1
        self.a = a
        self.b = b
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, a: {self.a}, b: {self.b}, x: {self.x}, y: {self.y}"


In [ ]:
def bissecao(
    f: Callable[[float], float],
    a: float,
    b: float,
    criterios: list[Callable[[EstatisticaAB], bool]],
    observador: Callable[[EstatisticaAB], any] = lambda _: None,
    max_iter=MAX_ITER_PADRAO,
):
    if a > b:
        a, b = b, a
    i = 1
    x = (a + b) / 2
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not all(map(lambda c: c(est), criterios)) and est.i < max_iter:
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a + b) / 2
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método da falsa posição

In [ ]:
def falsa_posicao(
    f: Callable[[float], float],
    a: float,
    b: float,
    criterios: list[Callable[[EstatisticaAB], bool]],
    observador: Callable[[EstatisticaAB], any] = lambda _: None,
    max_iter=MAX_ITER_PADRAO,
):
    if a > b:
        a, b = b, a
    i = 1
    x = (a * f(b) - b * f(a)) / (f(b) - f(a))
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not all(map(lambda c: c(est), criterios)) and est.i < max_iter:
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a * f(b) - b * f(a)) / (f(b) - f(a))
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método do ponto fixo

In [ ]:
class EstatisticaX:
    def __init__(
        self,
        f: Callable[[float], float],
        fi: Callable[[float], float],
        x: float,
        y: float,
    ) -> None:
        self.i: int = 1
        self.f = f
        self.fi = fi
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, x: float, y: float):
        self.i += 1
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, x: {self.x}, y:{self.y}"


In [ ]:
def ponto_fixo(
    f: Callable[[float], float],
    fi: Callable[[float], float],
    x: float,
    criterios: list[Callable[[EstatisticaX], bool]],
    observador: Callable[[EstatisticaX], any] = lambda _: None,
    max_iter=MAX_ITER_PADRAO,
):
    i = 1
    y = f(x)
    est = EstatisticaX(f, fi, x, y)
    observador(est)
    while not all(map(lambda c: c(est), criterios)) and est.i < max_iter:
        i += 1
        x = fi(x)
        y = f(x)
        est.atualizar(x, y)
        observador(est)
    return est


## Método de Newtown-Raphson

In [ ]:
def newton_raphson(
    f: Callable[[float], float],
    x: float,
    criterios: List[Callable[[EstatisticaX], bool]],
    observador: Callable[[EstatisticaX], any] = lambda x: None,
    max_iter=MAX_ITER_PADRAO,
):
    d = derivada_central(f)
    fi = lambda x: x - f(x) / d(x)
    return ponto_fixo(f, fi, x, criterios, observador, max_iter)


# Método de Gauss-Seidel

In [5]:
def compara(x,xk,precisao):
    soma = 0
    zip_object = zip(x, xk)
    for list1_i, list2_i in zip_object:
        soma = soma + math.fabs(list1_i - list2_i)

    if (soma < precisao):
        return True
    else:
        return False



def gaussSeidel(a, b, maxIter, precisao):
    n= len(b)
    solucao = True
    x = b.copy()
    for i in list(range(1, n+1, 1)):
        if((a[i-1] [i-1]) > 0.0):
            x[i-1] = b[i-1]/a[i-1][i-1]
        else:
            solucao = False
            break

        if(solucao):
            print("iteração 0")
            print("x =", x)
            xk = x.copy()
           # maxIter = 10
           # precisao = 0.01
            iter = 0

            while(iter < maxIter):
                iter = iter + 1
                for i in list(range(1, n+11, 1)):
                    s = 0
                    for j in list(range(1, n+1, 1)):
                        if((i-1) > (j-1)):
                            s = s + a[i-1][j-1] * xk[j-1]
                        elif ((i-1) < (j-1)):
                            s = s + a[i-1][j-1] * x[j-1]
                             
                    xk[i-1] = (1/a[i-1][i-1]) * (b[i-1] - s)

        return x


a = [[9, -2, 1, 3, 1],
     [1, -1, 2, 6, 5],
     [2, -3, 7, 1, 0],
     [1, 6, -1, 4, 2],
     [1, -2, 2, 1, 8]]

b = [0, 2, 1, 1, 2]

x = gaussSeidel(a, b, 10, 0.0001)

iteração 0
x = [0.0, 2, 1, 1, 2]


IndexError: list index out of range

# Amostras

In [ ]:
def amostra_linear(x):
    return 5 * x + 2


def amostra_poli(x):
    return x**2 + x - 6


def amostra_poli_iter(x):
    return sqrt(6 - x)


# Playground 

In [ ]:
resultado = ponto_fixo(
    amostra_poli, derivada_central(amostra_poli), 5, [criterio_x(0.003)], print
)


In [ ]:
somatorio = i*2

for i in range(n)